## Deployment

To deploy our APP, we need to first create a two Dockerfiles, one for frontend, and one for back end. The Dockerfiles should be located at the root dir of frontend and backend.

The docker file for frontend:

In [ ]:
# Multi-stage Dockerfile for FastAPI backend
FROM python:3.12-slim as base

# Set environment variables
ENV PYTHONPATH=/app
ENV PYTHONDONTWRITEBYTECODE=1
ENV PYTHONUNBUFFERED=1

# Install system dependencies
RUN apt-get update && apt-get install -y \
    curl \
    && rm -rf /var/lib/apt/lists/*

# Create app directory
WORKDIR /app

# Copy requirements first for better caching
COPY requirements.txt .

# Development stage
FROM base as development
RUN pip install --no-cache-dir -r requirements.txt #install python dependencies
COPY . .
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000", "--reload"] #launch the backend FaastAPI (we used to do it manually)

# Production stage
FROM base as production
RUN pip install --no-cache-dir -r requirements.txt
COPY . .

# Create non-root user
RUN addgroup --system --gid 1001 appgroup && \
    adduser --system --uid 1001 --gid 1001 --no-create-home appuser

# Change ownership of the app directory
RUN chown -R appuser:appgroup /app
USER appuser

# Health check
HEALTHCHECK --interval=30s --timeout=10s --start-period=10s --retries=3 \
    CMD curl -f http://localhost:8000/health || exit 1

# Expose port
EXPOSE 8000

# Start the application
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]

The Dockerfile for frontend:

In [ ]:
# Multi-stage Dockerfile for Next.js frontend
FROM node:20-alpine AS base

# Install dependencies only when needed
FROM base AS deps
# Check https://github.com/nodejs/docker-node/tree/b4117f9333da4138b03a546ec926ef50a31506c3#nodealpine to understand why libc6-compat might be needed.
RUN apk add --no-cache libc6-compat
WORKDIR /app

# Install dependencies based on the preferred package manager
COPY package.json package-lock.json* ./
RUN npm ci

# Rebuild the source code only when needed
FROM base AS builder
WORKDIR /app
COPY --from=deps /app/node_modules ./node_modules
COPY . .

# Build the application
RUN npm run build

# Production image, copy all the files and run next
FROM base AS runner
WORKDIR /app

ENV NODE_ENV production
ENV NEXT_TELEMETRY_DISABLED 1

RUN addgroup --system --gid 1001 nodejs
RUN adduser --system --uid 1001 nextjs

# Copy the built application
COPY --from=builder /app/public ./public
COPY --from=builder --chown=nextjs:nodejs /app/.next/standalone ./
COPY --from=builder --chown=nextjs:nodejs /app/.next/static ./.next/static

# Install wget for health checks
RUN apk add --no-cache wget

USER nextjs

EXPOSE 3000

ENV PORT 3000
ENV HOSTNAME "0.0.0.0"

# Health check
HEALTHCHECK --interval=30s --timeout=10s --start-period=10s --retries=3 \
    CMD wget --no-verbose --tries=1 --spider http://localhost:3000 || exit 1

CMD ["node", "server.js"]

We then need a docker compose configuration file (docker-compose.yml) added at the root of the app (under test_app); this will initialized the container for both backend and frontend and check whether they are working as expected:

In [ ]:
services:
  backend: 
    build:
      context: ./backend
      dockerfile: Dockerfile #the docker file we used to build container for backend
      target: production
    container_name: signal-lab-backend
    ports:
      - "8000:8000" # this is the default port for our backend
    environment:
      - PYTHONPATH=/app #the python working directoiry in Docker container
      - PYTHONUNBUFFERED=1
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:8000/health"] #check the health of connection/request by sending a request to heath endpoint
      interval: 30s
      timeout: 10s #if no response from backend API health endpoint, test fail
      retries: 3
      start_period: 10s
    restart: unless-stopped
    networks:
      - signal-lab-network

  frontend:
    build:
      context: ./frontend
      dockerfile: Dockerfile # Dockerfile of frontend
      target: runner
    container_name: signal-lab-frontend
    ports:
      - "3000:3000" #default port of the front end
    environment:
      - NODE_ENV=production
      - NEXT_TELEMETRY_DISABLED=1
      - NEXT_PUBLIC_API_URL=http://localhost:8000 #the environment variable we add to link frontend to backend port
    depends_on:
      backend:
        condition: service_healthy
    healthcheck:
      test: ["CMD", "wget", "--no-verbose", "--tries=1", "--spider", "http://localhost:3000"]
      interval: 30s
      timeout: 10s
      retries: 3
      start_period: 10s
    restart: unless-stopped
    networks:
      - signal-lab-network

networks:
  signal-lab-network:
    driver: bridge

We finally need a docker-start.sh file and a docker-end.sh files, so that we can start our APP in Docker containers using command line: 

docker-start.sh example:

In [ ]:
#!/bin/bash

# Resolve script directory
Resolve script directory
SCRIPT_DIR="$(cd "$(dirname "${BASH_SOURCE[0]}")" && pwd)"

# Project root is one level above scripts/
PROJECT_ROOT="$(dirname "$SCRIPT_DIR")"

# Move to project root (where docker-compose.yml is)
cd "$PROJECT_ROOT"

echo "🚀 Building Docker images..."
docker compose build

echo "🚀 Starting all services..."
docker compose up -d

echo "✅ All services started!"
docker compose ps

docker-end-sh example:

In [ ]:
#!/bin/bash

SCRIPT_DIR="$(cd "$(dirname "${BASH_SOURCE[0]}")" && pwd)"
PROJECT_ROOT="$(dirname "$SCRIPT_DIR")"
cd "$PROJECT_ROOT"

echo "🛑 Stopping all services..."
docker compose down

echo "🧹 Cleaning unused Docker resources (optional)..."
docker system prune -f

echo "✅ All services stopped!"


Remember to make these bash scripts executable:

In [ ]:
chmod +x scripts/docker-start.sh
chmod +x scripts/docker-end.sh  

We can then run the app through bash command: (run at root dir of the project)

In [ ]:
./scripts/docker-start.sh #start the app
./scripts/docker-end.sh #end the app

More importantly, with all the files listed above, we can share our APP to anyone who has Docker and they can run this app on any platform in the Docker container.